In [1]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import copy

In [2]:
# ucitavanje podataka
X = pd.read_csv("../heart_disease_uci.csv")

In [3]:
X

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,917,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
X=X.drop_duplicates()
X = X[(X['dataset'] == 'Cleveland') | (X['dataset'] == 'Switzerland')]
X.drop(('id'), axis=1, inplace=True)

In [5]:
X = X.loc[X.restecg.notnull(), :]

In [6]:
X2=X.copy()
#inace sam dropo ca, thal i +slope+ 
#X2.drop(['ca', 'thal', 'slope'], axis=1, inplace=True)
X2.drop(['dataset'], axis=1, inplace=True)
X2.loc[X2.chol.isna(),'chol'] = X2.loc[:,'chol'].mean()
X2.loc[X2.trestbps.isna(),'trestbps'] = X2.loc[:,'trestbps'].mean()
X2.loc[X2.thalch.isna(),'thalch'] = X2.loc[:,'thalch'].mean()
X2.loc[X2.oldpeak.isna(),'oldpeak'] = X2.loc[:,'oldpeak'].mean()

X2 = X2.loc[X2.ca.notnull(), :]
X2 = X2.loc[X2.thal.notnull(), :]
X2 = X2.loc[X2.slope.notnull(), :]
X3 = X2.loc[X2.fbs.notnull(), :]
X4 = X3.loc[X3.exang.notnull(), :]

X5 = X4.copy()
# X5['oldpeak'] = X5['oldpeak'].abs()
X5 = X5[X5['oldpeak'] >= 0]
# X5['chol']=X5['chol'].replace(0,X5['chol'].mean())
# X5['trestbps']=X5['trestbps'].replace(0,X5['trestbps'].mean())
X5 = X5[(X5['chol'] != 0) & (X5['trestbps'] != 0)]
X5 = X5[(X5['chol'] >= 126) & (X5['chol'] <= 564)]

In [7]:
le = LabelEncoder()
le.fit(X.loc[:,'restecg'])
X5.loc[:,'restecg'] = le.transform(X5.loc[:,'restecg'])

le = LabelEncoder()
le.fit(X.loc[:,'exang'])
X5.loc[:,'exang'] = le.transform(X5.loc[:,'exang'])

le = LabelEncoder()
le.fit(X.loc[:,'fbs'])
X5.loc[:,'fbs'] = le.transform(X5.loc[:,'fbs'])

le = LabelEncoder()
le.fit(X.loc[:,'cp'])
X5.loc[:,'cp'] = le.transform(X5.loc[:,'cp'])

le = LabelEncoder()
le.fit(X.loc[:,'ca'])
X5.loc[:,'ca'] = le.transform(X5.loc[:,'ca'])

le = LabelEncoder()
le.fit(X.loc[:,'thal'])
X5.loc[:,'thal'] = le.transform(X5.loc[:,'thal'])

le = LabelEncoder()
le.fit(X.loc[:,'slope'])
X5.loc[:,'slope'] = le.transform(X5.loc[:,'slope'])
# X5 = pd.get_dummies(X5, columns=['sex'], prefix='sex', drop_first=True)
# X5 = pd.get_dummies(X5, columns=['restecg'], prefix='restecg', drop_first=True)
# X5 = pd.get_dummies(X5, columns=['exang'], prefix='exang', drop_first=True)
# X5 = pd.get_dummies(X5, columns=['fbs'], prefix='fbs', drop_first=True)
# X5 = pd.get_dummies(X5, columns=['cp'], prefix='cp', drop_first=True)

#one hot 
X5 = pd.get_dummies(X5, columns=['sex'], prefix='sex', drop_first=False)

# X5 = pd.get_dummies(X5, columns=['dataset'], prefix='dataset', drop_first=False)

/var/folders/gz/blq0wwnn6bg59fmygpb2zpp80000gn/T/ipykernel_6436/359611030.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X5.loc[:,'restecg'] = le.transform(X5.loc[:,'restecg'])
/var/folders/gz/blq0wwnn6bg59fmygpb2zpp80000gn/T/ipykernel_6436/359611030.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X5.loc[:,'exang'] = le.transform(X5.loc[:,'exang'])
/var/folders/gz/blq0wwnn6bg59fmygpb2zpp80000gn/T/ipykernel_6436/359611030.py:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace

In [8]:
X=X5.copy()
y=X['num'].values
X=X.drop(labels=['num'], axis=1)
X

,age,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,sex_Female,sex_Male
0,63,3,145.0,233.0,1,0,150.0,0,2.3,0,0,0,0,1
1,67,0,160.0,286.0,0,0,108.0,1,1.5,1,3,1,0,1
2,67,0,120.0,229.0,0,0,129.0,1,2.6,1,2,2,0,1
3,37,2,130.0,250.0,0,1,187.0,0,3.5,0,0,1,0,1
4,41,1,130.0,204.0,0,0,172.0,0,1.4,2,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,140.0,241.0,0,1,123.0,1,0.2,1,0,2,1,0
298,45,3,110.0,264.0,0,1,132.0,0,1.2,1,0,2,0,1
299,68,0,144.0,193.0,1,1,141.0,0,3.4,1,2,2,0,1
300,57,0,130.0,131.0,0,1,115.0,1,1.2,1,1,2,0,1


In [9]:
for index,v in enumerate(y):
    if v > 1:
        y[index] = 1

In [10]:
X

,age,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,sex_Female,sex_Male
0,63,3,145.0,233.0,1,0,150.0,0,2.3,0,0,0,0,1
1,67,0,160.0,286.0,0,0,108.0,1,1.5,1,3,1,0,1
2,67,0,120.0,229.0,0,0,129.0,1,2.6,1,2,2,0,1
3,37,2,130.0,250.0,0,1,187.0,0,3.5,0,0,1,0,1
4,41,1,130.0,204.0,0,0,172.0,0,1.4,2,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,140.0,241.0,0,1,123.0,1,0.2,1,0,2,1,0
298,45,3,110.0,264.0,0,1,132.0,0,1.2,1,0,2,0,1
299,68,0,144.0,193.0,1,1,141.0,0,3.4,1,2,2,0,1
300,57,0,130.0,131.0,0,1,115.0,1,1.2,1,1,2,0,1


In [11]:
# from sklearn import svm
# from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from sklearn.model_selection import KFold, GridSearchCV
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, auc, roc_curve, ConfusionMatrixDisplay, RocCurveDisplay
# import matplotlib.pyplot as plt


# num_folds=10
# kf = StratifiedKFold(n_splits=num_folds, random_state=42, shuffle=True)
# X_np = X.to_numpy()

# nb = Pipeline([('scaler', MinMaxScaler()), ('classifier', GaussianNB())])
# rf = Pipeline([('scaler', MinMaxScaler()), ('classifier', RandomForestClassifier())])
# svm = Pipeline([('scaler', MinMaxScaler()), ('classifier', SVC())])
# lr = Pipeline([('scaler', MinMaxScaler()), ('classifier', LogisticRegression())])
# dt = Pipeline([('scaler', MinMaxScaler()), ('classifier', DecisionTreeClassifier())])

# classifiers = [nb, rf, svm, lr, dt]

# for classifier in classifiers:
#     total_accuracy = 0
#     total_recall = 0
#     total_precision = 0
#     total_confusion_matrix = np.zeros((2, 2))  # Assuming binary classification
#     total_roc_auc = 0

#     # Define parameter grid for grid search
#     param_grid = {}

#     if classifier == nb:
#         param_grid = {'classifier__var_smoothing': [1e-20, 1e-12, 1e-10, 1e-9, 1e-8, 1e-7,1e-5,1e-1,1, 10, 100]}
#     elif classifier == rf:
#         param_grid = {'classifier__n_estimators': [50, 100, 200],
#                       'classifier__max_depth': [None, 10, 20],
#                       'classifier__min_samples_split': [2, 5, 10],
#                       'classifier__random_state': [42]}
#     elif classifier == svm:
#         param_grid = {'classifier__C': [1e-4,1e-2,1e-1,1,10,50,100],
#                       'classifier__kernel': ['linear', 'rbf', 'poly'],
#                       'classifier__gamma': ['scale', 'auto'],
#                       'classifier__tol': [1e-10, 1e-9, 1e-6, 1e-4, 0.001, 0.01],
#                       'classifier__random_state': [42]}
#     elif classifier == lr:
#         param_grid = {'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 200, 500],
#                       'classifier__tol': [1e-10, 1e-9, 1e-6, 1e-4, 0.001, 0.01],
#                       'classifier__penalty': ['l2'],
#                       'classifier__solver': ['lbfgs', 'liblinear', 'newton-cholesky'],
#                       'classifier__random_state': [42]}
#     elif classifier == dt:
#         param_grid = {'classifier__max_depth': [None, 5, 10, 20, 25, 30, 40, 100],
#                       'classifier__min_samples_split': [2, 5, 10, 15, 20, 30],
#                       'classifier__min_samples_leaf': [1, 2, 3, 4, 6, 8],
#                       'classifier__splitter': ['best', 'random'],
#                       'classifier__criterion': ['gini', 'entropy', 'log_loss'],
#                       'classifier__random_state': [42]}

#     # Perform K-Fold cross-validation with grid search
#     grid_search = GridSearchCV(classifier, param_grid, cv=kf)
    
#     for train_index, test_index in kf.split(X_np, y):
#         X_train, X_test = X_np[train_index], X_np[test_index]
#         y_train, y_test = y[train_index], y[test_index]

#         # Perform grid search on the training data
#         grid_search.fit(X_train, y_train)

#         # Use the best model to make predictions on the test set
#         y_pred = grid_search.best_estimator_.predict(X_test)
        
#         # Evaluate accuracy
#         accuracy = accuracy_score(y_test, y_pred)
#         total_accuracy += accuracy
        
#         # Evaluate recall and precision
#         report = classification_report(y_test, y_pred, output_dict=True)
#         total_recall += report['1']['recall']
#         total_precision += report['1']['precision']

#         # Confusion matrix
#         total_confusion_matrix += confusion_matrix(y_test, y_pred)
        
#         # ROC-AUC
#         roc_auc = roc_auc_score(y_test, y_pred)
#         total_roc_auc += roc_auc
        
#     # Calculate average accuracy across folds
#     avg_accuracy = round(total_accuracy / num_folds, 4)
#     avg_recall = round(total_recall / num_folds, 4)
#     avg_precision = round(total_precision / num_folds, 4)
#     avg_confusion_matrix = np.around(total_confusion_matrix / num_folds, decimals=4)
#     avg_roc_auc = round(total_roc_auc / num_folds, 4)
    
#     # Print the results for each classifier
#     print(f"{classifier['classifier'].__class__.__name__} - Best Parameters: {grid_search.best_params_}")
#     print(f"{classifier['classifier'].__class__.__name__} - Average Accuracy: {avg_accuracy}")
#     print(f"{classifier['classifier'].__class__.__name__} - Average Recall: {avg_recall}")
#     print(f"{classifier['classifier'].__class__.__name__} - Average Precision: {avg_precision}")
#     print(f"{classifier['classifier'].__class__.__name__} - Average Confusion Matrix:\n{avg_confusion_matrix}")
#     print(f"{classifier['classifier'].__class__.__name__} - Average ROC-AUC: {avg_roc_auc}")
    
#     # Plot Confusion Matrix
#     plt.figure(figsize=(5, 5))
#     sns.heatmap(avg_confusion_matrix, annot=True, fmt=".2f", cmap="Blues", xticklabels=['0', '1'], yticklabels=['0', '1'])
#     plt.xlabel('Predictions')
#     plt.ylabel('Ground Truth')
#     plt.title(f"{classifier['classifier'].__class__.__name__} Confusion matrix")
#     plt.show()
    
#     RocCurveDisplay.from_estimator(grid_search, X_test, y_test)
#     plt.show()

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np

num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
X_np = X.to_numpy()

params = {
    "n_estimators": [50, 100, 200, 400],
    "learning_rate": [0.1, 0.01, 0.001, 0.15, 0.05],
    "max_depth": [1, 2, 3],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

clf = GradientBoostingClassifier(random_state=42)

accuracies = []
recalls = []
precisions = []

for train_index, test_index in kfold.split(X_np, y):
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y[train_index], y[test_index]

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    search = GridSearchCV(clf,
                          param_grid=params,
                          cv=3,
                          verbose=1,
                          n_jobs=1,
                          return_train_score=True
                          )

    search.fit(X_train_scaled, y_train)

    best_clf = search.best_estimator_
    best_clf.fit(X_train_scaled, y_train)

    y_pred = best_clf.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    accuracies.append(accuracy)
    recalls.append(recall)
    precisions.append(precision)

print("Average accuracy:", round(np.mean(accuracies), 4))
print("Average recall:", round(np.mean(recalls), 4))
print("Average precision:", round(np.mean(precisions), 4))


Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Average accuracy: 0.8216
Average recall: 0.7922
Average precision: 0.8183
